<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
import wikipediaapi
import re

def detect_language(text):
    """
    Detect whether the text is in Korean or English.
    """
    if re.search(r'[가-힣]', text):
        return 'ko' 
    else:
        return 'en' 

def extract_core_question(question, core_extraction_model):
    """
    Use LLM to extract the core part of the question, with prompts in the appropriate language.
    """
    language = detect_language(question)
    
    if language == 'ko':
        prompt = f"다음 질문에서 핵심 내용을 추출하세요 (선택지 제외):\n\n{question}\n\n핵심 내용:"
    elif language == 'en':
        prompt = f"Extract the core part of the following question (exclude choices):\n\n{question}\n\nCore content:"
    else:
        raise ValueError("Unsupported language detected.")
    core = core_extraction_model(prompt)
    return core.strip()

def initialize_wikipedia(language):
    """
    Initialize Wikipedia API object with the specified language.
    """
    return wikipediaapi.Wikipedia(
        language=language,
        user_agent='NLP_Team/1.0 (ewhanthbeot@ewhain.net)' 
    )

def fetch_from_wikipedia(query):
    """
    Fetch a summary from Wikipedia based on the query's language.
    """
    language = detect_language(query) 
    wiki_api = initialize_wikipedia(language) 
    
    page = wiki_api.page(query)
    if page.exists():
        return page.summary[:500] 
    else:
        return f"Wikipedia에서 관련 정보를 찾을 수 없습니다 (언어: {language})."


In [59]:
def normalize_answer(answer):
    """
    Normalize the format of the answer to handle cases like 'A' vs '(A)'.
    """
    if answer.startswith("(") and answer.endswith(")"):
        return answer[1:-1]  # Remove surrounding parentheses
    return answer.strip()  # Remove any extra whitespace

In [ ]:
correct = 0

for i, question in enumerate(prompts):
    core_question = extract_core_question(question, qa_chain["llm"])
    print(f"질문 핵심 추출: {core_question}")
    
    context = retrieve_context(core_question, list_kb)
    
    if not context.strip():
        print(f"PDF에서 관련 정보를 찾을 수 없습니다. Wikipedia를 검색합니다.")
        context = fetch_from_wikipedia(core_question)
    
    if not context.strip():
        print("PDF 및 Wikipedia 모두에서 관련 정보를 찾을 수 없습니다.")
        context = "No relevant information found."

    prompt = qa_chain["prompt"].format(context=context, question=question)
    
    result = qa_chain["llm"](prompt)
    predicted_answer = extract_answer(result)
    
    if predicted_answer == "N/A":
        print(f"답변이 N/A로 표시됨: 다시 확인 중...")
        prompt_with_choices = f"다음 선택지 중에서 정답을 선택하세요:\n{question}"
        result = qa_chain["llm"](prompt_with_choices)
        predicted_answer = extract_answer(result)

    normalized_predicted = normalize_answer(predicted_answer)
    normalized_actual = normalize_answer(answers[i])
    
    print(f"질문 {i + 1}: {question}")
    print(f"생성된 답변: {result}")
    print(f"예상 답: {normalized_predicted}, 실제 답: {normalized_actual}\n")
    
    if normalized_predicted == normalized_actual:
        correct += 1

질문 핵심 추출: 건축학전공의 수업연한은 5년입니다.
질문 1: QUESTION1) 건축학전공의 수업연한은?
(A) 4년
(B) 5년
(C) 6년
(D) 7년
생성된 답변: (C) 6년
예상 답: C, 실제 답: B

질문 핵심 추출: 학사경고를 받는 학점은 1.8입니다.
질문 2: QUESTION2) 학사경고를 받는 학점은?
(A) 1.6
(B) 1.8
(C) 2.0
(D) 2.2
생성된 답변: (C) 2.0
예상 답: C, 실제 답: A

질문 핵심 추출: 계절학기 취득 가능 학점은.
질문 3: QUESTION3) 계절학기 취득 가능 학점은?
(A) 2학점
(B) 4학점
(C) 6학점
(D) 8학점
생성된 답변: (C) 6학점
예상 답: C, 실제 답: C

질문 핵심 추출: 중앙행정기관이 아닌 기관은 교무처입니다.
답변이 N/A로 표시됨: 다시 확인 중...
질문 4: QUESTION4) 중앙행정기관이 아닌 기관은?
(A) 교무처
(B) 인재개발처
(C) 학생처
(D) 총무처
생성된 답변: 정답은 (A) 교무처입니다. 중앙행정기관은 일반적으로 정부 조직 내에서 주요 업무를 담당하는 기관을 의미합니다. (B) 인재개발처, (C) 학생처, (D) 총무처는 교육 기관 내의 주요 행정 기능을 수행하는 부서로 볼 수 있습니다. 반면, (A) 교무처는 주로 학사 행정과 교육 관련 업무를 담당하는 부서로, 중앙행정기관으로는 분류되지 않습니다.
예상 답: A, 실제 답: B

질문 핵심 추출: 체육특기자의 대회출전 및 훈련으로 인한 출석대체인정은 1학기 수업시간의 6분의 1 이내로 한다.
질문 5: QUESTION5) 체육특기자의 대회출전 및 훈련으로 인한 출석대체인정은 1학기 수업시간의 ___ 이내로 한다.
(A) 8분의 1
(B) 6분의 1
(C) 4분의 1
(D) 2분의 1
생성된 답변: (A) 8분의 1.
예상 답: A, 실제 답: D

질문 핵심 추출: 교육과목의 종류와 학점을 정하는 사람은 학장입니다.
질문 6: QUESTION6) 교육과목의 종류와 학점을 

STEP 5. 평가

In [61]:
# main 코드

In [62]:
# STEP 5. 평가
accuracy = correct / len(prompts) * 100
print(f"acc: {accuracy:.2f}%")

acc: 65.00%
